In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
def reduce_mem_usage(df):
    #iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    #create a dataframe and optimize its memory usage
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [ ]:
train_df = import_data('../input/tabular-playground-series-dec-2021/train.csv')
test_df = import_data('../input/tabular-playground-series-dec-2021/test.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()
print(" ")
train_df.isnull().sum()


In [ ]:
test_df.head()

In [ ]:
test_df.info()
print(" ")
test_df.isnull().sum()

In [ ]:
X = train_df.drop(['Id', 'Cover_Type'], axis=1)
y = train_df.Cover_Type

In [ ]:
test = test_df.drop(['Id'], axis=1)

In [ ]:
import imblearn
print(imblearn.__version__)
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler()
X_over, y_over = oversample.fit_resample(X, y)

In [ ]:
'''
from xgboost import XGBClassifier

xgb = XGBClassifier()

model = xgb.fit(X, y)
'''

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier()

model = lgb.fit(X, y)

In [ ]:
final_preds = model.predict(test)

In [ ]:
from sklearn.metrics import mean_absolute_error, r2_score


print("Mean absolute error: {}".format(mean_absolute_error(y[:1000000], final_preds)))
print("r2 score: {}".format(r2_score(y[:1000000], final_preds)))

In [ ]:
sample = pd.read_csv('../input/tabular-playground-series-dec-2021/sample_submission.csv')

In [ ]:
submission = sample.copy()

submission['Cover_Type'] = final_preds

submission

In [ ]:
submission.to_csv('submission.csv', index=False)